<a href="https://colab.research.google.com/github/VenkateshSoni/Fake-News-Classifier-using-LSTM/blob/main/Kaggle_Fake_New_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fake News Classifier Using LSTM**

Dataset: https://www.kaggle.com/c/fake-news/data#

Getting the Dataset

In [42]:
#install Kagglea
!pip install -q Kaggle

In [43]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"venkateshsoni","key":"9900cfc58e2c768d8dfa0ca35a87785b"}'}

In [44]:
#create a kaggle folder
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [45]:
#copy tge Kaggle.json to folder created
! cp kaggle.json ~/.kaggle/

In [46]:
# Persmissioon for the json to act
! chmod 600 ~/.kaggle/kaggle.json

In [47]:
#Listing all datasets in Kaggle
! kaggle datasets list

ref                                                            title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/piterfm/2022-ukraine-russian-war                      2022 Ukraine Russia War                            2KB  2022-04-13 07:24:12           7681        434  1.0              
datasets/kamilpytlak/personal-key-indicators-of-heart-disease  Personal Key Indicators of Heart Disease           3MB  2022-02-16 10:18:03          12439        270  1.0              
datasets/ankanhore545/cost-of-living-index-2022                Cost of Living Index 2022                          4KB  2022-04-09 06:22:10           2550         45  0.8235294        
datasets/prasertk/best-tv-shows-of-all-time                    Best TV shows of 

In [48]:
!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [49]:
!unzip fake-news.zip

Archive:  fake-news.zip
replace submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: submit.csv              
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


Implementing LSTM

Take Titles -> Apply One Hot Representation -> Pass through the Embedding Layer -> Pass to LSTM -> Train the model -> Predict the test data

In [105]:
import pandas as pd

In [106]:
df=pd.read_csv(r'/content/train.csv')

In [107]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [108]:
#dropping nan values  
df = df.dropna()

In [109]:
#Get the independent features
X = df.drop('label', axis=1)

In [110]:
#get the dependent features
y = df['label']

In [111]:
X.shape

(18285, 4)

In [112]:
y.shape

(18285,)

In [113]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [114]:
from tensorflow.keras.layers import Embedding #Embedding layer
from tensorflow.keras.preprocessing.sequence import pad_sequences #Pre and post pad sequences to keep input legth fixed
from tensorflow.keras.models import Sequential #Sequential Model
from tensorflow.keras.preprocessing.text import one_hot #converting sentences into one hot given a vocabulary size
from tensorflow.keras.layers import LSTM #Layer
from tensorflow.keras.layers import Dense #Dense layer 

In [115]:
#Vocabulary size
voc_size = 5000

One Hot Representation For Title messages(can try for text column too)

In [116]:
messages = X.copy()

In [117]:
messages.reset_index(inplace=True)

In [118]:
import nltk
import re
from nltk.corpus import stopwords #for data preprocessing

In [119]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [120]:
#Data preprocessing
from nltk.stem.porter import PorterStemmer #for stemming
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [121]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [122]:
onehot_representation = [one_hot(words, voc_size) for words in corpus]
onehot_representation

[[1609, 1, 1954, 1754, 63, 2222, 1373, 51, 2962, 2313],
 [4420, 604, 2688, 2266, 3764, 4286, 2889],
 [481, 790, 771, 1318],
 [1588, 3973, 1881, 4475, 2363, 4012],
 [1701, 3764, 339, 4025, 3084, 4791, 3764, 3539, 3466, 2785],
 [3862,
  2307,
  171,
  3531,
  3070,
  4391,
  2722,
  4983,
  3889,
  3184,
  97,
  4541,
  4385,
  3519,
  2889],
 [2216, 3938, 2017, 3992, 1528, 2304, 4892, 1821, 2927, 2470, 4462],
 [4270, 435, 4066, 3520, 2144, 4730, 4391, 454, 2927, 2470, 4462],
 [3055, 2282, 1798, 2060, 478, 3907, 919, 4872, 4391, 2905],
 [3827, 158, 153, 2396, 4313, 866, 1816, 4274],
 [3159, 3934, 1269, 1762, 3968, 4982, 2766, 3128, 3251, 3829, 3186],
 [4475, 1206, 63, 3907, 4391, 2144],
 [4158, 515, 4498, 2540, 2618, 3619, 704, 1070, 3022],
 [2213, 3159, 2520, 1434, 131, 29, 4517, 2927, 2470, 4462],
 [3148, 4879, 3189, 2258, 343, 2927, 2470, 4462],
 [3329, 469, 1401, 3635, 4289, 2710, 4250, 2617, 718, 1734],
 [1473, 9, 604],
 [4383, 4278, 884, 1969, 4391, 2970, 3299, 2889],
 [4982, 4903,

Embedding Representation --for making abocve sentences of same length by adding padding

In [123]:
sent_length = 20 
embedded_docs = pad_sequences(onehot_representation, padding = 'pre', maxlen = sent_length)
print(embedded_docs) 

[[   0    0    0 ...   51 2962 2313]
 [   0    0    0 ... 3764 4286 2889]
 [   0    0    0 ...  790  771 1318]
 ...
 [   0    0    0 ... 2927 2470 4462]
 [   0    0    0 ... 2035  318  488]
 [   0    0    0 ... 1092 2206 1504]]


In [124]:
len(embedded_docs)

18285

In [125]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4420,  604, 2688, 2266, 3764, 4286, 2889], dtype=int32)

In [126]:
#Creating Model
embedding_vector_features = 40 #features
model = Sequential() #creating sequential layer
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100)) #1 LSTM layer with 100 Neurons
model.add(Dense(1, activation = 'sigmoid')) #As classification problem
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy']) 
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [127]:
len(embedded_docs), y.shape

(18285, (18285,))

In [128]:
import numpy as np 
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [129]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [130]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

**Model** **Training**

In [131]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 28ms/step - loss: 0.3384 - accuracy: 0.8470 - val_loss: 0.1962 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 5s 25ms/step - loss: 0.1408 - accuracy: 0.9438 - val_loss: 0.1907 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0991 - accuracy: 0.9631 - val_loss: 0.2303 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0717 - accuracy: 0.9743 - val_loss: 0.2514 - val_accuracy: 0.9171
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0564 - accuracy: 0.9809 - val_loss: 0.3116 - val_accuracy: 0.9017
Epoch 6/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0405 - accuracy: 0.9861 - val_loss: 0.3282 - val_accuracy: 0.9176
Epoch 7/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0273 - accuracy: 0.9913 - val_loss: 0.3707 - val_accuracy: 0.9157

**Performance, Metrics and Accuracy**

In [132]:
y_pred = (model.predict(X_test) > 0.5).astype("int64") #model.predict_classes(X_test) --depriciated

In [133]:
from sklearn.metrics import confusion_matrix

In [134]:
confusion_matrix(y_test,y_pred)

array([[3097,  322],
       [ 184, 2432]])

In [135]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9161557580778791

Adding Dropout Layer - Hyperparameter

In [136]:
from tensorflow.keras.layers import Dropout
#Creating Model
embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])